In [1]:
#! /usr/bin/env python3.5
import tensorflow as tf
import numpy as np

import model.cnn as cnn
import model.classifier as classifier

import os, sys
from scipy import io

flags = tf.app.flags
flags.DEFINE_integer('gpu', 0, 'device to train on [0]')
FLAGS = flags.FLAGS
FLAGS._parse_flags()

os.environ['CUDA_VISIBLE_DEVICES'] = str(FLAGS.gpu)

# ResNet

### Load recording

In [2]:
#fname = sys.argv[1]
dir = '../validation/'
fname = 'A00001'
assert os.path.isfile(dir + fname + ".mat"), "Not existing file: " + dir + fname + ".mat"
data = io.loadmat(dir + fname + '.mat')['val'].astype(np.float32).squeeze()
data -= data.mean()
data /= data.std()

### Set up predictor

In [3]:
print('Building model graph...')
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(1, [], name='batch_size')

input_op = tf.placeholder(tf.float32, [1, None])
seq_len = tf.placeholder(tf.float32, [1])

cnn_params = {
    'out_dims' : [32, 64, 64],
    'kernel_sizes' : 64,
    'pool_sizes' : 1
}

c = cnn.model(
    seq_len=seq_len, 
    input_op=input_op, 
    model_name='CNN_block',
    **cnn_params)

RESIDUAL_POOL = 4
residual_input = c.output[..., None, :]

for i in range(1, 4):    
    residual_input = tf.contrib.layers.max_pool2d(
        residual_input, 
        kernel_size=[RESIDUAL_POOL, 1], 
        stride=[RESIDUAL_POOL, 1])
    
    c = cnn.model(
        seq_len=seq_len, 
        input_op=residual_input, 
        residual=True, 
        model_name='CNN_block_%d'%i,
        **cnn_params)
    residual_input += c.output

res_out = tf.squeeze(residual_input, axis=2)
a = tf.reduce_mean(res_out, axis=1)
fc = classifier.model(input_op=a, fc_sizes=[16])

pred = fc.pred

Building model graph...
CNN_block--cnn32x64-64x64-64x64
Tensor("CNN_block/Conv1/dropout/mul:0", shape=(1, ?, 1, 32), dtype=float32)
Tensor("CNN_block/Conv2/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
Tensor("CNN_block/Conv3/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
CNN_block_1--cnn32x64-64x64-64x64
Tensor("CNN_block_1/Conv1/dropout/mul:0", shape=(1, ?, 1, 32), dtype=float32)
Tensor("CNN_block_1/Conv2/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
Tensor("CNN_block_1/Conv3/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
CNN_block_2--cnn32x64-64x64-64x64
Tensor("CNN_block_2/Conv1/dropout/mul:0", shape=(1, ?, 1, 32), dtype=float32)
Tensor("CNN_block_2/Conv2/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
Tensor("CNN_block_2/Conv3/dropout/mul:0", shape=(1, ?, 1, 64), dtype=float32)
CNN_block_3--cnn32x64-64x64-64x64
Tensor("CNN_block_3/Conv1/dropout/mul:0", shape=(1, ?, 1, 32), dtype=float32)
Tensor("CNN_block_3/Conv2/dropout/mul:0", shape=(1, ?, 1, 64), dtype

### Run predictor

In [4]:
label_dict = {0: 'N', 1: 'A', 2: 'O', 3: '~'}
saver = tf.train.Saver()
with tf.Session() as sess:
    print('Sess started')
    coord = tf.train.Coordinator()
    saver.restore(
        sess,
        '../entry/v2/model/pool4--cnn32x64-64x64-64x64--fc16-20000')
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print('Evaluating')
    output = sess.run(pred, feed_dict={input_op: [data],
                                       seq_len: [len(data)],
                                       batch_size: 1})

    print('Closing threads')
    coord.request_stop()
    coord.join(threads)

    result = label_dict[np.where(tf.equal(output, tf.reduce_max(output, axis=1)[:, None]).eval()[0])[0][0]]

Sess started
Evaluating
Closing threads


In [6]:
print(output)

[[  1.06540145e-02   2.91159451e-01   6.98186517e-01   1.23510473e-08]]


# ConvNet

### Load recording

In [2]:
#fname = sys.argv[1]
dir = '../validation/'
fname = 'A00001'
assert os.path.isfile(dir + fname + ".mat"), "Not existing file: " + dir + fname + ".mat"
data = io.loadmat(dir + fname + '.mat')['val'].astype(np.float32).squeeze()
data -= data.mean()
data /= data.std()

### Set up predictor

In [3]:
print('Building model graph...')
tf.reset_default_graph()
batch_size = tf.placeholder_with_default(1, [], name='batch_size')

input_op = tf.placeholder(tf.float32, [1, None])
seq_len = tf.placeholder(tf.float32, [1])

cnn_params = {
    'out_dims': [128, 256, 256],
    'kernel_sizes': 64,
    'pool_sizes': 1,
    'model_name': 'CNN'
}
c = cnn.model(seq_len=seq_len, input_op=input_op, **cnn_params)

a = tf.reduce_mean(c.output, axis=1)
fc = classifier.model(input_op=a, fc_sizes=[])

pred = fc.pred

Building model graph...
CNN--cnn128x64-256x64-256x64
Tensor("CNN/Conv1/dropout/mul:0", shape=(1, ?, 1, 128), dtype=float32)
Tensor("CNN/Conv2/dropout/mul:0", shape=(1, ?, 1, 256), dtype=float32)
Tensor("CNN/Conv3/dropout/mul:0", shape=(1, ?, 1, 256), dtype=float32)

FC--fc
Tensor("classifier/logits/BiasAdd:0", shape=(1, 4), dtype=float32)
Tensor("classifier/predictions:0", shape=(1, 4), dtype=float32)


### Run predictor

In [4]:
label_dict = {0: 'N', 1: 'A', 2: 'O', 3: '~'}
saver = tf.train.Saver()
with tf.Session() as sess:
    print('Sess started')
    coord = tf.train.Coordinator()
    saver.restore(
        sess,
        '../entry/v2/model/--cnn128x64-256x64-256x64--fc-46000')
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print('Evaluating')
    output = sess.run(pred, feed_dict={input_op: [data],
                                       seq_len: [len(data)],
                                       batch_size: 1})

    print('Closing threads')
    coord.request_stop()
    coord.join(threads)

    result = label_dict[np.where(tf.equal(output, tf.reduce_max(output, axis=1)[:, None]).eval()[0])[0][0]]

Sess started
Evaluating
Closing threads


### Save result

In [5]:
print(result)

N


In [7]:
with open("answers.txt", "a") as file:
    file.write(fname + ',' + result)